In [1]:
import os
import numpy as np
import torchaudio
# from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
# import torch
from tqdm import tqdm

In [2]:
# # Initialize the Wav2Vec2 processor and model
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Define the root directory where your folders are located
root_dir = r"tensorflow-speech-recognition-challenge\train\audio"

# Initialize lists to store audio data and labels
all_audio_data = []
all_labels = [label for label in os.listdir(root_dir)]
all_labels = all_labels[:5]

In [35]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
		
# load dummy dataset and read soundfiles
ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

# tokenize
input_values = processor(ds[0]["audio"]["array"], return_tensors="pt", padding="longest").input_values  # Batch size 1

# Pass the preprocessed input through the Wav2Vec2 model to get the vectors
with torch.no_grad():
	vectors = model(input_values).last_hidden_state
 
# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Thilina\Documents\Projects\FYP - Cleansing the Digital Landscape\.venv\lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for patrickvonplaten/librispeech_asr_dummy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/patrickvonplaten/librispeech_asr_dummy
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors t

AttributeError: 'CausalLMOutput' object has no attribute 'last_hidden_state'

In [30]:
audios[0][0]

tensor([ 9.1553e-05,  3.0518e-05,  1.8311e-04,  ..., -3.0518e-05,
        -9.1553e-05,  1.2207e-04])

In [29]:
ds[0]["audio"]["array"]

array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
       0.0010376 ])

In [21]:
input_values.shape

torch.Size([1, 93680])

In [3]:
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import torch

c:\Users\Thilina\Documents\Projects\FYP - Cleansing the Digital Landscape\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
audios = []
y = []

In [7]:
# Iterate through each folder in the root directory
for label in all_labels:
	print(f"{label = }")
	label_dir = os.path.join(root_dir, label)
	# Iterate through each audio file in the folder
	count = 1
	for audio_file in tqdm(os.listdir(label_dir)):
		if count == 100:
			continue
		else:
			count += 1
		audio_path = os.path.join(label_dir, audio_file)
		try:
			waveform, _ = torchaudio.load(audio_path)
			audios.append(waveform)
			y.append(label)
		except Exception as e:
			print(f"Error loading audio file '{audio_path}': {e}")

# # Extract features
# inputs = processor(audios, sampling_rate=16_000, padding="longest", return_tensors="pt")
# with torch.no_grad():
# 	embeddings = model(**inputs).logits


label = 'bed'


100%|██████████| 1713/1713 [00:00<00:00, 35731.98it/s]


label = 'bird'


100%|██████████| 1731/1731 [00:00<00:00, 37912.01it/s]


label = 'cat'


100%|██████████| 1733/1733 [00:00<00:00, 32768.00it/s]


label = 'dog'


100%|██████████| 1746/1746 [00:00<00:00, 32018.99it/s]


label = 'down'


100%|██████████| 2359/2359 [00:00<00:00, 28549.06it/s]


In [8]:
max_len = max([waveform.shape[1] for waveform in audios])
max_len

16000

In [9]:
# Pad shorter audios with silence
padded_audios = []

In [10]:
 # Create a sample of silence
for audio in audios:
	padding_len = max_len - audio.shape[1]
	padded_audio = torch.nn.functional.pad(audio, (0, padding_len), mode='constant', value=0.0)
	padded_audios.append(padded_audio)

In [11]:
# Load the pre-trained processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

# List to store the processed vectors
all_vectors = []

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for padded_audio in padded_audios:
	# Preprocess the audio waveform
	input_values = processor(padded_audio[0], return_tensors="pt", padding=True).input_values

In [33]:
input_values[0]

tensor([-0.1162, -0.1043, -0.0980,  ...,  0.1179,  0.1219,  0.1324])

In [34]:
for padded_audio in padded_audios:
	# Preprocess the audio waveform
	input_values = processor(padded_audio[0], return_tensors="pt", padding=True).input_values
	
	# Pass the preprocessed input through the Wav2Vec2 model to get the vectors
	with torch.no_grad():
		vectors = model(input_values).last_hidden_state
	
	# Convert the vectors to a NumPy array and append to the list
	vectors_array = vectors.squeeze().numpy()
	all_vectors.append(vectors_array)

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


AttributeError: 'CausalLMOutput' object has no attribute 'last_hidden_state'

In [7]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch.nn.functional as F

In [ ]:
# # Desired fixed length for all waveforms
# max_length = 16000  # Adjust as needed

# padded_audios = []
# for waveform in audios:
#     # Calculate the amount of padding needed
#     pad_length = max_length - waveform.size(0)
#     if pad_length > 0:
#         # Pad the waveform with zeros at the end
#         padded_waveform = F.pad(waveform, (0, pad_length))
#     elif pad_length < 0:
#         # Truncate the waveform to max_length
#         padded_waveform = waveform[:max_length]
#     else:
#         # No padding or truncation needed
#         padded_waveform = waveform
    
#     padded_audios.append(padded_waveform)

# Convert the list of padded waveforms to a tensor
padded_audios_tensor = torch.stack(padded_audios)

# Now you can use StandardScaler with the padded waveforms
scaler = StandardScaler()
features_scaled = scaler.fit_transform(padded_audios_tensor)

In [8]:
# Now you can use StandardScaler with the padded waveforms
scaler = StandardScaler()
padded_audios_tensor = torch.stack(padded_audios)
padded_audios_tensor = padded_audios_tensor.reshape(padded_audios_tensor.shape[0], -1)  # Reshape to (n_audios, features)
features_scaled = scaler.fit_transform(padded_audios_tensor.numpy())  # Convert to numpy array for scikit-learn

In [ ]:
# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, y, test_size=0.2, random_state=42)

In [11]:
from sklearn.svm import SVC

In [12]:
# Initialize SVM classifier
svm_model = SVC(probability=True, random_state=42)

In [13]:
# Train the model
svm_model.fit(X_train, y_train)
# 100*5 -> 32.2s
# 200*5 -> 2m 34.1s
# 400*5 -> 11m 44.0s

SVC(probability=True, random_state=42)

In [14]:
# Make predictions
y_pred = svm_model.predict(X_test)
y_prob = svm_model.predict_proba(X_test)

In [15]:
from sklearn.metrics import classification_report

In [16]:
# Evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         bed       0.00      0.00      0.00        83
        bird       0.12      0.03      0.04        75
         cat       0.20      0.87      0.32        70
         dog       0.21      0.09      0.12        94
        down       0.22      0.08      0.12        77

    accuracy                           0.19       399
   macro avg       0.15      0.21      0.12       399
weighted avg       0.15      0.19      0.12       399



100*5 -> 32.2s______=> accuracy    0.20

200*5 -> 2m 34.1s__=> accuracy    0.21

400*5 -> 11m 44.0s_=> accuracy    0.19

In [21]:
y_test[1]

'cat'

In [22]:
# Example of predicting probabilities for a new audio sample
# new_audio_features = extract_features(new_audio_file)
scaled_audio_features = scaler.transform(X_test[1].reshape(1, -1))
probabilities = svm_model.predict_proba(scaled_audio_features)
print(probabilities)

[[0.16350958 0.2128641  0.13664375 0.25499349 0.23198907]]
